# 번역서 03.4 부분의 4. memory_key_padding_mask 내용은 아래 코드에서 Let's Declare Model 부분의 일부를 참조하세요.

In [ ]:
# 원서의 깃허브에서 제공하는 코드는 원서의 내용보다 훨씬 방대합니다.
# 이에 따라, 원서 본문 및 번역번 본문에서는 깃허브 제공 코드의 일부분만 설명하고 있습니다.
# 이는 타 챕터에도 적용됩니다.

In [ ]:
# 원서 번역 시점에서 호환되는 torch 및 torchtext 조합 설치
# 이 다음 코드 불록에서 필요함
# 런타임 3분 소요
!pip install torch==2.0.1+cu118 torchtext==0.15.2 -f https://download.pytorch.org/whl/torch_stable.html

# 이 코드를 실행 후 코랩 메뉴 [런타임] > [세션 다시 시작](restart the session)을 실행해야 함

In [ ]:
!pip install transformers
!pip install datasets
!pip install torch
!pip install torchtext

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


#역자 추가 코드

In [ ]:
import torch
import torch.nn as nn

# 랜덤 시드 설정
torch.manual_seed(0)

# 파라미터 정의
num_embeddings = 10 # 어휘(vocabulary) 개수
embedding_dim = 3   # 임베딩 벡터 차원

# 임베딩 층
embedding = nn.Embedding(
    num_embeddings=num_embeddings, embedding_dim=embedding_dim)
input_tokens = torch.tensor([1, 5])
output_embeddings = embedding(input_tokens)
print(output_embeddings)

tensor([[-0.4339,  0.8487,  0.6920],
        [-0.1116, -0.6136,  0.0316]], grad_fn=<EmbeddingBackward0>)


#Text Classification

In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
def get_device():
  device="cpu"
  if torch.cuda.is_available():
    device="cuda"
  elif  torch.backends.mps.is_available():
    device='mps'
  else:
    device="cpu"
  return device


device = get_device()
print(device)

cuda


In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, dim_embedding, dropout=0.1, max_seq_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        postional_encoding = torch.zeros(max_seq_len, dim_embedding)
        position = torch.arange(0, max_seq_len, dtype=torch.float).unsqueeze(1)
        denom_term = torch.exp(torch.arange(0, dim_embedding, 2).float() * (-math.log(10000.0) / dim_embedding))
        postional_encoding[:, 0::2] = torch.sin(position * denom_term)
        postional_encoding[:, 1::2] = torch.cos(position * denom_term)
        postional_encoding = postional_encoding.unsqueeze(0).transpose(0, 1)
        self.register_buffer('postional_encoding', postional_encoding)
    def forward(self, x):
        x = x + self.postional_encoding[:x.size(0), :]
        return self.dropout(x)

## Data Preparation:
Here's our plan for data preparation.
Prepare the IMDb dataset for text classification:

1. Load the IMDb dataset and tokenize the text using an appropriate pre-trained tokenizer.
2.Pad and truncate the tokenized text to ensure a consistent sequence length across different examples.
3. Ensure the data shapes are compatible with the TransformerEncoder requirements:
  * Reshape the tokenized data (input_ids) to have a shape of (seq_length, batch_size).
  * Reshape the attention mask to have a shape of (batch_size, seq_length).
  * One-hot encode the labels for binary-class classification.

4. Create TensorDataset to use the huggingface data on Pytorch.
  * TensorDataset is a utility class in PyTorch (from the torch.utils.data module) that allows you to create a dataset object by wrapping one or more tensors. Each tensor in the dataset represents a different field or attribute of your data samples, such as input data, labels, or attention masks.
5. Define custom Collate_fn
  * The default collate function in DataLoader simply combines the samples into a batch without any additional processing. However, in many cases, you may need to perform custom processing on your samples
  * Accorind to our shape requirement. Performe Shape transformation in this function.
  * data: (seq_len, batch_size)
  * atten_mask: (batch_size, seq_len)
  * labe: one-hot-encode

Important Point: Understand the Attention Mask:

  * The attention mask is a binary tensor that indicates which tokens in the input sequence should be attended to by the model.
In the attention mask tensor, the value 1 corresponds to an actual token (word or subword), while the value 0 corresponds to a padding token.
  * The model uses this mask to ignore padding tokens during the self-attention mechanism in the Transformer architecture.

Let's Code it!

In [ ]:
# 런타임 30초~2분 소요
from datasets import load_dataset
from transformers import AutoTokenizer
from torch.utils.data import TensorDataset, DataLoader
import torch

# 데이터셋과 토크나이저 불러오기
dataset = load_dataset("imdb")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# 데이터셋 토큰화
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, return_tensors="pt", max_length=512)

train_dataset = dataset["train"].map(tokenize, batched=True, batch_size=len(dataset["train"]))
val_dataset = dataset["test"].map(tokenize, batched=True, batch_size=len(dataset["test"]))

# 토큰화된 데이터셋에서 input_ids와 attention_mask 추출
train_data = torch.tensor(train_dataset["input_ids"])
train_attention_mask = torch.tensor(train_dataset["attention_mask"])
train_labels = torch.tensor(train_dataset["label"])

val_data = torch.tensor(val_dataset["input_ids"])
val_attention_mask = torch.tensor(val_dataset["attention_mask"])
val_labels = torch.tensor(val_dataset["label"])

# TensorDataset 생성
train_dataset = TensorDataset(train_data, train_attention_mask, train_labels)
val_dataset = TensorDataset(val_data, val_attention_mask, val_labels)

# DataLoader 생성
def collate_fn(batch):
    input_ids, attention_mask, labels = zip(*batch)
    input_ids = torch.stack(input_ids).transpose(0, 1) # input_ids 트랜스포즈(Transpose)
    attention_mask = torch.stack(attention_mask)       # attention_mask 트랜스포즈(Transpose)
    labels = torch.nn.functional.one_hot(torch.tensor(labels), num_classes=2).float().to(device)
    return input_ids, attention_mask, labels

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

The src_key_padding_mask in the TransformerEncoder should be a 2D boolean tensor of shape (batch_size, sequence_length). Each element in the tensor should be True if the corresponding token in the input sequence is a padding token, and False otherwise.

The src_key_padding_mask is used to mask out the padding tokens in the input so that they don't contribute to the attention calculation. When the padding tokens are masked out, the transformer will not consider them while computing attention scores for the non-padding tokens.

This is our idea for Text Classification
1. Initialize the following components in the constructor:
  * Embedding layer to convert input tokens into embeddings.
  * Positional encoding to add position information to the embeddings.
  * Transformer encoder consisting of multiple layers, each with multi-head self-attention and feedforward neural networks.
  * Fully connected (linear) layer for classification.
2. Implement the init_weights method to initialize the weights of the model components.

3. Implement the forward method to define the forward pass of the model.
  * Pass the input through the embedding layer and apply positional encoding. We also add multiplication term to the embedding. The purpose of this multiplication is to scale the embeddings. This can help the model learn better and avoid vanishing gradients. The square root of the model's dimension is used as a scaling factor because it is a simple heuristic that works well in practice.
  * Pass the embeddings through the transformer encoder with an optional key_padding_mask.
  * Perform mean pooling on the last dimension and use the first token representation.
    1. The output of encoder is (Seq_length, batch_size, emb_dim). Performing the mean polling across the dimesion `0` will change the data in the form of (batch_size, emb_dim)
  * Pass the pooled representation through the fully connected layer for classification.
  * Apply a sigmoid activation function to obtain probabilities.

In [ ]:
class TextClassifier(nn.Module):
    def __init__(
        self, vocab_size, embedding_dim, nhead, num_layers, num_classes):
        super(TextClassifier, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.positional_encoding = PositionalEncoding(embedding_dim)
        # 트랜스포머 인코더 층 생성
        self.encoder_layer = nn.TransformerEncoderLayer(
            embedding_dim, nhead)
        self.encoder = nn.TransformerEncoder(
            self.encoder_layer, num_layers)
        self.fc = nn.Linear(embedding_dim, num_classes)
        self.embedding_dim=embedding_dim
        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        self.embedding.weight.data.uniform_(-initrange, initrange)
        for layer in self.encoder.layers:
            nn.init.xavier_uniform_(layer.self_attn.out_proj.weight)
            nn.init.zeros_(layer.self_attn.out_proj.bias)
            nn.init.xavier_uniform_(layer.linear1.weight)
            nn.init.zeros_(layer.linear1.bias)
            nn.init.xavier_uniform_(layer.linear2.weight)
            nn.init.zeros_(layer.linear2.bias)
        self.fc.bias.data.zero_()
        self.fc.weight.data.uniform_(-initrange, initrange)

    def forward(self, x, key_padding_mask=None):
        x = self.embedding(x)* math.sqrt(self.embedding_dim)
        x = self.positional_encoding(x)
        x = self.encoder(x, src_key_padding_mask=key_padding_mask)

        # 첫 번째 차원을 기준으로 나머지 차원(마지막 차원) 값의 평균값 생성
        x = x.mean(dim=0)

        # 분류 작업용 완전 연결 층
        x = self.fc(x)
        x=torch.sigmoid(x)
        return x

## Declaring Model

In [ ]:
import torch.optim as optim  # optim 모듈 임포트 추가
import torch.nn as nn

vocab_size = tokenizer.vocab_size
embedding_dim = 512
nhead = 8
num_layers = 6
num_classes = 2

# 모델 생성
model = TextClassifier(vocab_size, embedding_dim, nhead, num_layers,  num_classes).to(device)
criterion = nn.BCELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

## Training
Here are a few important things to note:

* batch_attention_mask = (batch_attention_mask == 0).to(device): Convert the attention mask to a boolean tensor by checking if the values are equal to 0.
  1. The attention mask that comes from Hugging Face tokenizer is as follows:
    * attention_mask = 1 if it is a real token
    * attention_mask = 0 if it is a pad token
  2. PyTorch's attention_mask requires us to send:
    * attention_mask = False if it is a real token
    * attention_mask = True if it is a pad token
  3. Therefore, we are converting batch_attention_mask == 0 to True. This means we are telling the model that if it is a pad token, set attention_mask = True.
* To avoid exploding gradients, clip the gradients of the model's parameters using torch.nn.utils.clip_grad_norm_(). This helps maintain the stability of the training process and prevents the gradients from becoming too large.


In [ ]:
# 런타임 9분~11분 소요
num_epochs = 1
for epoch in range(num_epochs):
    i=0
    for batch_data, batch_attention_mask, batch_labels in train_dataloader:

        optimizer.zero_grad()

        # attention_mask를 불리언(boolean) 텐서로 변환
        batch_attention_mask = (batch_attention_mask==0).to(device)

        outputs = model(batch_data.to(device), key_padding_mask=batch_attention_mask)
        loss = criterion(outputs, batch_labels.to(device))
        if i%100==0:
          print ("epoch ", epoch, "batch ", i, "loss ", loss)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
        i=i+1

    print(f"Epoch: {epoch + 1}, Loss: {loss.item()}")

epoch  0 batch  0 loss  tensor(0.8624, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
epoch  0 batch  100 loss  tensor(0.7319, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
epoch  0 batch  200 loss  tensor(0.7225, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
epoch  0 batch  300 loss  tensor(0.6896, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
epoch  0 batch  400 loss  tensor(0.6924, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
epoch  0 batch  500 loss  tensor(0.7088, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
epoch  0 batch  600 loss  tensor(0.7117, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
epoch  0 batch  700 loss  tensor(0.6897, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
Epoch: 1, Loss: 0.6769706010818481


In [ ]:
# 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 런타임 20초 소요
# 독자 여러분의 구글 드라이브의 알맞은 폴더에 저장
torch.save(model.state_dict(), "/content/drive/MyDrive/Book6/Ch3/TextClassificationModel.pth")

## Inference


In [ ]:
# 토크나이저 초기화
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
vocab_size = tokenizer.vocab_size
embedding_dim = 512
nhead = 8
num_layers = 6 #원서 코드에서는 3으로 잘못 기재되어 고침
num_classes = 2

# 모델 생성
model_loaded = TextClassifier(vocab_size, embedding_dim, nhead, num_layers,  num_classes).to(device)

# 학습 모델 가중치(weights) 불러오기
model_loaded.load_state_dict(torch.load('/content/drive/MyDrive/Book6/Ch3/TextClassificationModel.pth'))
model.eval()

TextClassifier(
  (embedding): Embedding(30522, 512)
  (positional_encoding): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder_layer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
    )
    (linear1): Linear(in_features=512, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=512, bias=True)
    (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-5): 6 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
     

### Printing total parameter of our model

In [ ]:
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total trainable parameters:", total_params)

Total trainable parameters: 37694978


In [ ]:

# 주어진 텍스트에서 추론을 실행하는 함수
def infer(text):
    # 입력 텍스트 토큰화
    tokens = tokenizer.encode_plus(text, padding=True, truncation=True, return_tensors="pt", max_length=512)
    input_ids = tokens["input_ids"].to(device).transpose(0,1)

    attention_mask = tokens["attention_mask"]
    attention_mask=(attention_mask==0).to(device)
    print(input_ids.shape)
    print(attention_mask)

    # 추론 실행
    with torch.no_grad():
        output = model(input_ids, key_padding_mask=attention_mask)
    # 출력을 클래스 확률로 변환
    probabilities = output.squeeze(0)
    return probabilities




In [ ]:
# 샘플 텍스트로 테스트
example_text = "This movie is  good! ."
probabilities = infer(example_text)

print("Probabilities:", probabilities)

torch.Size([8, 1])
tensor([[False, False, False, False, False, False, False, False]],
       device='cuda:0')
Probabilities: tensor([0.5042, 0.5029], device='cuda:0')


Summary:
The provided code offers a basic implementation of a TransformerEncoder for text classification. The primary goal is to demonstrate how to utilize a TransformerEncoder for this task. In subsequent chapters, we will explore optimized versions of Transformer-based classification. To achieve better results with the current code, consider increasing the number of encoder layers and fine-tuning various parameters.

# Text Generation
## Decoder-Only Layer
1. We will use the Shakespeare dataset to create a decoder-only model that generates text in the style of Shakespeare.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


What we are doing Here?
1. This code defines a custom PyTorch dataset called ShakespeareDataset.
2. It takes a file path to a text file and a tokenizer as input. The dataset reads the file and splits it into examples of size block_size.
3. Each example is then tokenized using the tokenizer and padded or truncated to a maximum length of block_size. The resulting tokenized examples are stored in a list.

In [ ]:
class ShakespeareDataset(Dataset):
    def __init__(self, file_path, tokenizer, block_size=128):
        self.block_size = block_size
        self.tokenizer = tokenizer

        with open(file_path, 'r') as f:
            self.data = f.read()

        self.examples = []
        for i in range(0, len(self.data)-self.block_size, self.block_size):
            example = self.data[i:i+self.block_size]
            tokenized = self.tokenizer(example, padding='max_length', truncation=True, max_length=block_size, return_tensors='pt')
            self.examples.append(tokenized)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        input_ids = self.examples[idx]['input_ids'].squeeze()
        attention_mask = self.examples[idx]['attention_mask'].squeeze()
        return input_ids, attention_mask




In [ ]:
import os,urllib
url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
# 독자 여러분의 알맞은 구글 드라이브 폴더로 대치
filename = '/content/drive/MyDrive/Book6/Ch3/input.txt'
if not os.path.isfile(filename):
    urllib.request.urlretrieve(url, filename)


In [ ]:
train_dataset = ShakespeareDataset(filename, tokenizer)

What we are doing here?
1. `inputs = torch.stack(inputs).transpose(0, 1)`: The dimension of input needed is (seq_length, batch_size). Thus, we are transposing.
2. `torch.stack(inputs)` takes a list of tensors and stacks them along a new dimension (the result has one more dimension than the input tensors).
  * For example, each inputs has dimension of (128). If the batch size 6; then, stacking operation will produce list of (6,128)=> (batch_size, seq_length)
  * transpose will result in (128,6)==> (seq_length, batch_size)

In [ ]:
from torch.nn.utils.rnn import pad_sequence
def collate_fn(batch):
    inputs, masks = zip(*batch)
    inputs = torch.stack(inputs).transpose(0, 1)
    masks = torch.stack(masks)
    return inputs, masks
train_dataloader = DataLoader(train_dataset, batch_size=4, collate_fn=collate_fn,shuffle=True)


In [ ]:
# 데이터의 차원 확인
item=next(iter(train_dataloader))
input_ids,attention_masks=item
print(input_ids.shape, attention_masks.shape)

torch.Size([128, 4]) torch.Size([4, 128])


## Let's Declare Positional Encoding Class.
1. It is same as what we discussed in earlier example

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, dim_embedding, dropout=0.1, max_seq_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        postional_encoding = torch.zeros(max_seq_len, dim_embedding)
        position = torch.arange(0, max_seq_len, dtype=torch.float).unsqueeze(1)
        denom_term = torch.exp(torch.arange(0, dim_embedding, 2).float() * (-math.log(10000.0) / dim_embedding))
        postional_encoding[:, 0::2] = torch.sin(position * denom_term)
        postional_encoding[:, 1::2] = torch.cos(position * denom_term)
        postional_encoding = postional_encoding.unsqueeze(0).transpose(0, 1)
        self.register_buffer('postional_encoding', postional_encoding)
    def forward(self, x):
        x = x + self.postional_encoding[:x.size(0), :]
        return self.dropout(x)

## Let's Declare Model
### Model Architecture
1. This model is a Transformer-based decoder-only language model, which takes as input a target sequence (tgt) and an  memory sequence (memory) and generates an output sequence of the same length as the input sequence.

2. The input target sequence is first passed through an embedding layer and a positional encoding layer. Similarly, the input memory sequence is passed through an embedding layer and a positional encoding layer.

3. During, Training
  * `memory` is train data of shape (seq_len, batch_size)
  * `target`:During model training, the target sequence would be the input sequence shifted by one position.

4. These processed input sequences are then fed into the Transformer decoder, which consists of multiple Transformer decoder layers. Each decoder layer processes the input sequences using multi-head self-attention and a feedforward neural network.

5. Finally, the output of the Transformer decoder is passed through a linear layer (fully-connected neural network) to generate the final output sequence, with each element of the sequence representing the probability distribution over the vocabulary of the target language.

In [ ]:
class TransformerDecoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_layers, dropout):
        super().__init__()

        self.memory_embedding = nn.Embedding(vocab_size, embedding_dim)
        self.memory_pos_encoder = PositionalEncoding(embedding_dim, dropout)
        self.tgt_embedding = nn.Embedding(vocab_size, embedding_dim)
        self.tgt_pos_encoder = PositionalEncoding(embedding_dim, dropout)
        self.decoder = nn.TransformerDecoder(
            nn.TransformerDecoderLayer(d_model=embedding_dim, nhead=8,
                                       dim_feedforward=2048,
                                       dropout=dropout),
            num_layers=num_layers)

        self.fc = nn.Linear(embedding_dim, vocab_size)
        self.d_model=embedding_dim
        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1

        # 임베딩 층 초기화
        nn.init.uniform_(self.memory_embedding.weight, -initrange, initrange)
        nn.init.uniform_(self.tgt_embedding.weight, -initrange, initrange)

        # 디코딩 층 초기화
        for param in self.decoder.parameters():
            if param.dim() > 1:
                nn.init.xavier_uniform_(param)

        # 출력 층 초기화
        nn.init.uniform_(self.fc.weight, -initrange, initrange)
        nn.init.zeros_(self.fc.bias)

    def forward(self, tgt,  memory=None, tgt_mask=None, memory_mask=None, memory_key_padding_mask=None,tgt_key_padding_mask=None):
        tgt = self.tgt_embedding(tgt) * self.d_model ** 0.5
        tgt=self.tgt_pos_encoder(tgt)
        print(tgt)
        memory=self.memory_embedding(memory) * self.d_model ** 0.5
        memory=self.memory_pos_encoder(memory)
        print(memory)
        output = self.decoder(
            tgt=tgt, memory=memory, tgt_mask=tgt_mask,
            memory_mask=memory_mask,
            memory_key_padding_mask=memory_key_padding_mask,
            tgt_key_padding_mask=tgt_key_padding_mask
            )
        print(output)
        output = self.fc(output)
        return output

# 번역서 03.4 부분의 4. memory_key_padding_mask 내용은 아래 코드에서 4 부분을 참조하세요.

Parameters  for decoding Layer

1. tgt: The input sequence to the decoder layer. It is a tensor of shape (seq_len, batch_size, emb_dim) where seq_len is the length of the input sequence and batch_size is the number of sequences in a batch.

2. memory: The output of the last layer of the encoder. It is a tensor of shape (src_seq_len, batch_size, emb_dim) where src_seq_len is the length of the input sequence in the encoder.

3. tgt_mask: An optional tensor of shape (seq_len, seq_len) representing the mask for the input sequence. It is used to prevent the decoder from attending to future tokens.
The format should be:

```
tensor([[0., -inf, -inf],
        [0., 0., -inf],
        [0., 0., 0.]], device='mps:0')
```
  * in above example, seq_length=3
  * where `-inf` signifies the tokens that need to be masked

4. memory_mask: 인코더 출력 시퀀스에 사용되는 마스크를 표기하는 (seq_len, src_seq_len) 형태의 옵션형 텐서입니다. 이는 디코더가 인코더 입력 시퀀스에 있는 미래 토큰들에 어텐션을 취하지 못하게 방지합니다.
(An optional tensor of shape (seq_len, src_seq_len) representing the mask for the encoder output sequence. It is used to prevent the decoder from attending future tokens in the encoder input sequence.)

```
tensor([[0., -inf, -inf],
        [0., 0., -inf],
        [0., 0., 0.]], device='mps:0')
```
  * 위의 예에서 seq_length=3 입니다.(in above example, seq_length=3)
  * `-inf`는 마스크가 필요한 토큰의 위치를 의미합니다. (where `-inf` signifies the tokens that need to be masked)

```
통상적으로, 여러분은 메모리를 마스크하지 않는 경우 다음과 같은 텐서를 사용합니다.
(Usually, you will not mask the memory: Thus, you will pass:)
tensor([[0., 0, 0],
        [0., 0., 0],
        [0., 0., 0.]], device='mps:0')

5. tgt_key_padding_mask: An optional tensor of shape (batch_size, seq_len) representing the mask for padding tokens in the input sequence.

```
tensor([[False, False, False],
        [False, False, False],
        [False, True, False],
        [True, True, False]], device='mps:0')
```
  * In above example, batch_size=4,  seq_len=3
  * True signifies the particular token is padded token and mask it
  * False signifies the particular token is padded token and mask it

6. memory_key_padding_mask: An optional tensor of shape (batch_size, src_seq_len) representing the mask for padding tokens in the encoder output sequence.

```
tensor([[False, False, False],
        [False, False, False],
        [False, True, False],
        [True, True, False]], device='mps:0')
```
  * In above example, batch_size=4,  seq_len=3
  * True signifies the particular token is padded token and mask it
  * False signifies the particular token is padded token and mask it


In [ ]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=device)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask.to(device)


def create_mask(src, tgt,tokenizer_src=tokenizer,tokenizer_tgt=tokenizer):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=device).type(torch.bool)

    src_padding_mask = (src == tokenizer_src.pad_token_id).transpose(0, 1)
    tgt_padding_mask = (tgt == tokenizer_tgt.pad_token_id).transpose(0, 1)
    return src_mask.to(device), tgt_mask.to(device), src_padding_mask.to(device), tgt_padding_mask.to(device)

In [ ]:
import math
#device='mps' # 코랩 사용시 불필요한 코드라서 주석 처리함
model = TransformerDecoder(vocab_size=tokenizer.vocab_size, embedding_dim=768, num_layers=3, dropout=0.1)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)


## Exercise
1. Write the Training Loop
2. Write the Inference Loop

# Transformer Layer
## Machine Translation
1. Machine Translation is the task of converting a text from one language to another. In this context, we will focus on English to German (en-de) translation. The task involves processing a sequence of tokens in one language and producing a corresponding sequence of tokens in another language.
2. We will use Encoder-Decoder Layer of Transformer




## Let's Download the Data

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from datasets import load_dataset

# 데이터셋과 토크나이저 불러오기
dataset = load_dataset("iwslt2017", "iwslt2017-de-en", split="train[:1%]")  # Only use a 1% portion of the dataset
tokenizer_src = AutoTokenizer.from_pretrained("bert-base-uncased")
tokenizer_tgt = AutoTokenizer.from_pretrained("bert-base-german-cased")

# 'Do you wish to run the custom code?'라는 질문이 결과 창에 나오면 소문자 y를 입력하고 엔터키를 누르면 됩니다.


README.md:   0%|          | 0.00/18.5k [00:00<?, ?B/s]

iwslt2017.py:   0%|          | 0.00/8.17k [00:00<?, ?B/s]

The repository for iwslt2017 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/iwslt2017.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


de-en.zip:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/206112 [00:00<?, ? examples/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/255k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/485k [00:00<?, ?B/s]

In [ ]:
dataset[0]

{'translation': {'de': 'Vielen Dank, Chris.',
  'en': 'Thank you so much, Chris.'}}

## Custom Dataset to Prepare Data
1. Let's Prepare the data so that we have src_tokem and tgt_token of same length
2. Here, we define the max length of token is 50

In [ ]:
class TranslationDataset(Dataset):
    def __init__(self, dataset, tokenizer_src, tokenizer_tgt, max_length=50):
        self.dataset = dataset
        self.tokenizer_src = tokenizer_src
        self.tokenizer_tgt = tokenizer_tgt
        self.max_length = max_length

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        src_text = self.dataset[idx]['translation']['en']
        tgt_text = self.dataset[idx]['translation']['de']

        src_tokens = self.tokenizer_src.encode_plus(
            src_text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )

        tgt_tokens = self.tokenizer_tgt.encode_plus(
            tgt_text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )

        return src_tokens["input_ids"].squeeze(),tgt_tokens["input_ids"].squeeze()


In [ ]:
train_data = TranslationDataset(dataset, tokenizer_src, tokenizer_tgt)


In [ ]:
# torch.tensor 생성 후 샘플 데이터 확인
train_data[2]

(tensor([  101,  1045,  2031,  2042, 10676,  2185,  2011,  2023,  3034,  1010,
          1998,  1045,  2215,  2000,  4067,  2035,  1997,  2017,  2005,  1996,
          2116,  3835,  7928,  2055,  2054,  1045,  2018,  2000,  2360,  1996,
          2060,  2305,  1012,   102,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]),
 tensor([    3,  1671,  4058,  4899, 15227,    88,   534, 13854, 26918,    42,
          1169,  9334, 26897,  9830,  2122,   142,    30,  2709,  2055,  2636,
             7, 18930,    81, 10183,  4468,  2085,  6738,  4253, 26914,     4,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]))

Analysis: If you look at the above sample, `0` represents the padding token

## Create DataLoader:
* The output after the dataloder should be of shape (seq_len, batch_size)
* Thus, we have transposed both src_ids, and tgt_ids

In [ ]:
def collate_fn(batch):
    src_ids ,tgt_ids = zip(*batch)
    src_ids = torch.stack(src_ids).transpose(0, 1)
    tgt_ids = torch.stack(tgt_ids).transpose(0, 1)
    return src_ids, tgt_ids
dataloader = DataLoader(train_data, batch_size=16, shuffle=True, collate_fn=collate_fn)


In [ ]:
item=next(iter(dataloader))
src_ids,tgt_ids=item
print('src_ids ',src_ids.shape)
print(' tgt_ids ',tgt_ids.shape)


src_ids  torch.Size([50, 16])
 tgt_ids  torch.Size([50, 16])


It looks perfect:
1. the shape of source and target are [seq_length, batch_size]


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext

class PositionalEncoding(nn.Module):
    def __init__(self, dim_embedding, dropout=0.1, max_seq_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        postional_encoding = torch.zeros(max_seq_len, dim_embedding)
        position = torch.arange(0, max_seq_len, dtype=torch.float).unsqueeze(1)
        denom_term = torch.exp(torch.arange(0, dim_embedding, 2).float() * (-math.log(10000.0) / dim_embedding))
        postional_encoding[:, 0::2] = torch.sin(position * denom_term)
        postional_encoding[:, 1::2] = torch.cos(position * denom_term)
        postional_encoding = postional_encoding.unsqueeze(0).transpose(0, 1)
        self.register_buffer('postional_encoding', postional_encoding)
    def forward(self, x):
        x = x + self.postional_encoding[:x.size(0), :]
        return self.dropout(x)

## Transformer Model
What we are doing?
1. `forward`
  * The source and target sequences are embedded and scaled by the square root of the embedding dimension.
  * The positional encodings are added to the embeddings.
  * The Transformer processes the source and target sequences, with masking
    1. src_mask, trg_mask==> This is done to prevent future flow of information
    2. src_padding_mask, trg_padding_mask ==> This is done to mask padded data. We are doing this so that model donot attent to padded tokens
  * The output of the Transformer is passed through a fully connected layer to get the predicted target sequence..
  * The model is predicting next token in german given all the tokens in en, and tokens untill the current step in german.
  

In [ ]:
class TransformerModel(nn.Module):
    def __init__(self,num_encoder_layers, num_decoder_layers, d_model,
                 nhead, src_vocab_size=tokenizer_src.vocab_size,
                 tgt_vocab_size=tokenizer_tgt.vocab_size,
                 dim_feedforward=512, dropout=0.1):
        super(TransformerModel, self).__init__()
        self.src_embedding = nn.Embedding(input_dim, d_model)
        self.trg_embedding = nn.Embedding(output_dim, d_model)
        self.src_pos_encoder = PositionalEncoding(d_model, dropout)
        self.trg_pos_encoder = PositionalEncoding(d_model, dropout)
        self.transformer = nn.Transformer(
            d_model=d_model, nhead=nhead,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dim_feedforward=dim_feedforward, dropout=dropout)
        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)
        self.d_model = d_model

    def forward(self, src, trg, src_mask=None,
                src_padding_mask=None, trg_mask=None,
                trg_padding_mask=None, memory_key_padding_mask=None):

        src = self.src_embedding(src) * (self.d_model ** 0.5)
        src = self.src_pos_encoder(src)
        trg = self.trg_embedding(trg) * (self.d_model ** 0.5)
        trg = self.trg_pos_encoder(trg)
        output = self.transformer(src, trg,src_mask, trg_mask, None,
                                  src_padding_mask, trg_padding_mask,
                                  memory_key_padding_mask)
        output = self.fc(self.dropout(output))
        return output

## Let's Creating Masking
1. `generate_square_subsequent_mask` will create following matrix for the tgt_msk. Here, we supposed the tgt sequence length is 3.

```
tensor([[ 0., -inf, -inf],
        [ 0.,  0., -inf],
        [ 0.,  0.,  0.]])
```


2. This means that each token would only be allowed to attend to the tokens that have already been generated during decoding.
3. `src_mask` square matrix fill with False. This means, we are not masking any source sequence
4. `src_padding_mask` and `tgt_padding_mask`: It looks at the src and tgt which is filled with padded token. And, mask all padded tokens
5. We transpose `src_padding_mask` and `tgt_padding_mask` because Transformer requires the dimension to be [batch_size, seq_len]. This is opposite to the Transformer requirement for src and tgt to be [seq_length, batch_size]


In [ ]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=device)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask.to(device)


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=device).type(torch.bool)

    src_padding_mask = (src == tokenizer_src.pad_token_id).transpose(0, 1)
    tgt_padding_mask = (tgt == tokenizer_tgt.pad_token_id).transpose(0, 1)
    return src_mask.to(device), tgt_mask.to(device), src_padding_mask.to(device), tgt_padding_mask.to(device)

In [ ]:
generate_square_subsequent_mask(3)

tensor([[0., -inf, -inf],
        [0., 0., -inf],
        [0., 0., 0.]], device='cuda:0')

## Let's Intiate Model and Conduct Training
1. `nn.CrossEntropyLoss(ignore_index=tokenizer_tgt.pad_token_id)`
  * We are asking loss function to ignore where it is padded token
2. `tgt_out = tgt[1:, :]`

  * we are removing the first token of the target sequence, since it corresponds to the special start-of-sentence token <sos>. By removing this token, we obtain a new tensor tgt_out that contains the remaining tokens of the target sequence, which will be used as input to the decoder during training. This is because during training, we want the model to learn to generate the target sequence given the input source sequence, without being provided with the start-of-sentence token

3. ` loss=criterion(output.reshape(-1, output.shape[-1]), tgt_out.reshape(-1))
`
  * The output tensor has shape [tgt_seq_len - 1, batch_size, tgt_vocab_size], which means it has 3 dimensions. To calculate the loss, we need to reshape it to a 2D tensor of shape [(tgt_seq_len - 1) * batch_size, tgt_vocab_size].

  * Similarly, tgt_out tensor has shape [tgt_seq_len - 1, batch_size], but to calculate the loss, we need to flatten it into a 1D tensor of shape [(tgt_seq_len - 1) * batch_size].

In [ ]:
### 역자 주: 다음 코드는 예시로서 지금까지 해 온 코드와는 입출력 차원이 맞지 않는 등 에러가 발생할 수 있습니다.
"""
input_dim = 50
output_dim=50
emb_size=512
nhead = 8
num_encoder_layers =num_decoder_layers= 3

model = TransformerModel(num_encoder_layers, num_decoder_layers, emb_size, nhead, src_vocab_size=tokenizer_src.vocab_size, tgt_vocab_size=tokenizer_tgt.vocab_size, dim_feedforward=512).to(device)

# Training loop
num_epochs = 1
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer_tgt.pad_token_id)
optimizer = optim.Adam(model.parameters())
model.train()
losses = 0
for epoch in range(num_epochs):
    for i, (src_ids, tgt_ids) in enumerate(dataloader):
        src=src_ids.to(device)
        tgt=tgt_ids.to(device)
        tgt_input = tgt[:-1, :]
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)
        # (self, src, trg, src_mask=None, src_padding_mask=None,trg_mask=None, trg_padding_mask=None, memory_key_padding_mask=None)
        output=model(src, tgt_input, src_mask, src_padding_mask,tgt_mask, tgt_padding_mask, src_padding_mask)
        optimizer.zero_grad()
        tgt_out = tgt[1:, :]
        loss=criterion(output.reshape(-1, output.shape[-1]), tgt_out.reshape(-1))
        loss.backward()
        optimizer.step()
        if i%100==0:
          print('epoch ', epoch, 'batch ', i, ' loss ', loss)
        losses=loss.item()
    print( losses / float(len(list(dataloader))))
"""


### Analysis:
The above code is a simplified version of a machine translation model. In future chapters, we will explore more advanced models.